In [1]:
import openml
import pandas as pd
import lightgbm as lgb
import catboost as cb
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


In [2]:
dataset = openml.datasets.get_dataset(31)

X, y, categorical_indicator, attribute_names = dataset.get_data(target=dataset.default_target_attribute, dataset_format='dataframe')

df = X
categorical_columns = [i for i, indicator in enumerate(categorical_indicator) if indicator]

label_encoder = LabelEncoder()
for col in categorical_columns:
    df.iloc[:, col] = label_encoder.fit_transform(df.iloc[:, col])

df

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,residence_since,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker
0,1,6,1,6,1169.0,4,3,4,3,2,4,3,67,1,1,2,1,1,1,1
1,0,48,3,6,5951.0,2,0,2,0,2,2,3,22,1,1,1,1,1,0,1
2,3,12,1,2,2096.0,2,1,2,3,2,3,3,49,1,1,1,3,2,0,1
3,1,42,3,3,7882.0,2,1,2,3,1,4,1,45,1,0,1,1,2,0,1
4,1,24,2,4,4870.0,2,0,3,3,2,4,2,53,1,0,2,1,2,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,3,12,3,3,1736.0,2,1,3,0,2,4,3,31,1,1,1,3,1,0,1
996,1,30,3,9,3857.0,2,0,4,1,2,4,1,40,1,1,1,0,1,1,1
997,3,12,3,6,804.0,2,3,4,3,2,4,0,38,1,1,1,1,1,0,1
998,1,45,3,6,1845.0,2,0,4,3,2,4,2,23,1,0,1,1,1,1,1


In [3]:
X

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,residence_since,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker
0,1,6,1,6,1169.0,4,3,4,3,2,4,3,67,1,1,2,1,1,1,1
1,0,48,3,6,5951.0,2,0,2,0,2,2,3,22,1,1,1,1,1,0,1
2,3,12,1,2,2096.0,2,1,2,3,2,3,3,49,1,1,1,3,2,0,1
3,1,42,3,3,7882.0,2,1,2,3,1,4,1,45,1,0,1,1,2,0,1
4,1,24,2,4,4870.0,2,0,3,3,2,4,2,53,1,0,2,1,2,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,3,12,3,3,1736.0,2,1,3,0,2,4,3,31,1,1,1,3,1,0,1
996,1,30,3,9,3857.0,2,0,4,1,2,4,1,40,1,1,1,0,1,1,1
997,3,12,3,6,804.0,2,3,4,3,2,4,0,38,1,1,1,1,1,0,1
998,1,45,3,6,1845.0,2,0,4,3,2,4,2,23,1,0,1,1,1,1,1


In [4]:
y

0      good
1       bad
2      good
3      good
4       bad
       ... 
995    good
996    good
997    good
998     bad
999    good
Name: class, Length: 1000, dtype: category
Categories (2, object): ['good' < 'bad']

In [5]:
y = y.replace({'good': 1, 'bad': 0})

In [6]:
# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the LightGBM model
lgb_model = lgb.LGBMClassifier()
lgb_model.fit(X_train, y_train)

# Evaluate the model
accuracy = lgb_model.score(X_test, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.8


In [9]:
# Train the CatBoost model
catboost_model = cb.CatBoostClassifier(verbose=False)
catboost_model.fit(X_train, y_train)

## Evaluate the CatBoost model
catboost_accuracy = catboost_model.score(X_test, y_test)
print("CatBoost Accuracy:", catboost_accuracy)


CatBoost Accuracy: 0.805


In [10]:
# Train the XGBoost model
xgboost_model = xgb.XGBClassifier()
xgboost_model.fit(X_train, y_train)

# Evaluate the XGBoost model
xgboost_accuracy = xgboost_model.score(X_test, y_test)
print("XGBoost Accuracy:", xgboost_accuracy)

XGBoost Accuracy: 0.835
